In [1]:
from models.pretrain.resnet_autoencoder import ResNet18Autoencoder
from data.autoencoding_dataset import AutoencoderDataset
from configs.pyroot_config import ConfigPaths
import torch
import torch.nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as tt
import numpy as np

In [2]:
# model = ResNet18Autoencoder()
# model.cuda()

# print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [3]:
#Reshape to 100 100

#compose transofmr receives a single channel pil image and converts it to a torch tensor
#crop image by 10 pixels from each side

transforms = tt.Compose([

    tt.ToTensor(),
    tt.Grayscale(num_output_channels=1),
    tt.Resize((80, 80)),
    tt.Lambda(lambda x: x[:, 16:]),
    tt.CenterCrop(64),
])


dataset_path = ConfigPaths.data_processed_pretrain

dataset = AutoencoderDataset(dataset_path, frames_dir='cropped_frames', transforms=transforms)
#dataset = torch.utils.data.Subset(dataset, range(128))

print(len(dataset))
sample = dataset[0]
    
dataloader = DataLoader(dataset, batch_size=512, shuffle=True, num_workers=20)



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


1116167


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [4]:
# print(type(dataset[0]))
# print(dataset[0].shape)

# #plot 30 consecutive images

# # fig, ax = plt.subplots(6, 5, figsize=(20, 20))
# # for i in range(30):
# #     mean = (dataset[i] - dataset[i+1]).abs().mean()
# #     std = (dataset[i] - dataset[i+1]).abs().std()
# #     print(mean, std)
# #     normalized_img = (dataset[i] - dataset[i+1] - mean) / std
# #     #0 1
# #     normalized_img = (normalized_img - normalized_img.min()) / (normalized_img.max() - normalized_img.min()) 

# #     ax[i//5, i%5].imshow(normalized_img.permute(1, 2, 0))

# #     # ax[i//5, i%5].imshow((dataset[i] - dataset[i+1]).permute(1, 2, 0))
# #     # print((dataset[i] - dataset[i+1]).min(), (dataset[i] - dataset[i+1]).max())
# #     ax[i//5, i%5].axis('off')

# fig, ax = plt.subplots(6, 5, figsize=(20, 20))

# for i in range(30):
#     ax[i//5, i%5].imshow((dataset[i*2+2000] - dataset[i*2+2002]).permute(1, 2, 0))
#     # print((dataset[i] - dataset[i+1]).min(), (dataset[i] - dataset[i+1]).max())
#     ax[i//5, i%5].axis('off')

# fig, ax = plt.subplots(6, 5, figsize=(20, 20))

# for i in range(30):
#     ax[i//5, i%5].imshow((dataset[i*2+2000]).permute(1, 2, 0))
#     # print((dataset[i] - dataset[i+1]).min(), (dataset[i] - dataset[i+1]).max())
#     ax[i//5, i%5].axis('off')

In [5]:
from models.pretrain.ae.resnet import ResNetEncoder, ResNetDecoder, get_configs, ResNetAutoEncoder
import torch

configs, bottleneck = get_configs("resnet34")
autoencoder = ResNetAutoEncoder(configs, bottleneck)
autoencoder.cuda()
#num params
print(sum(p.numel() for p in autoencoder.parameters() if p.requires_grad))
model = autoencoder

42561024


In [7]:
#take 30 random numbers between 0 and 1116167
random_indices = np.random.randint(0, 1116167, 10)


loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):



    for i, data in tqdm(enumerate(dataloader)):
        data = data.cuda()
        optimizer.zero_grad()
        outputs = model(data)
        loss = loss_fn(outputs, data)
        loss.backward()
        optimizer.step()


        if i % 30 == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")
            model.eval()
            fig, axs = plt.subplots(2, 10, figsize=(25, 5))
            for j, idx in enumerate(random_indices):
                data = dataset[idx].unsqueeze(0).cuda()
                outputs = model(data)
                axs[0, j].imshow(data.detach().cpu().numpy().squeeze(), cmap='gray')
                axs[1, j].imshow(outputs.detach().cpu().numpy().squeeze(), cmap='gray')
                axs[0, j].axis('off')
                axs[1, j].axis('off')
            plt.savefig(f'./reports/pretrain/plots/epoch_{epoch}_batch_{i}.png')
            #close figure 
            plt.close(fig)
            model.train()
    torch.save(model.state_dict(), f"./models/pretrain/ae/resnet/epoch_{epoch}.pt")



0it [00:00, ?it/s]

Epoch: 0, Batch: 0, Loss: 0.001588910585269332


30it [00:20,  1.83it/s]

Epoch: 0, Batch: 30, Loss: 0.0010039489716291428


60it [00:37,  1.82it/s]

...

Epoch: 0, Batch: 1650, Loss: 0.0003528370289131999


1680it [18:29,  1.58it/s]

Epoch: 0, Batch: 1680, Loss: 0.00021210385602898896


1701it [18:44,  1.51it/s]


KeyboardInterrupt: 